In [1]:
import pandas as pd
import xarray as xr
import pathlib
import json

## Ingested Data

In [2]:
# ingested dataset where cell id is int

cell_tidy_data = pd.read_hdf(
    '/home/hanliu/pkg/omb/omb/Data/Dataset/Variables.h5')

def read_msgpack(path):
    with open(path, 'rb') as f:
        data = msgpack.unpackb(f.read())
    return data
cell_to_int = read_msgpack('/home/hanliu/pkg/omb/omb/Data/Dataset/CellIDMap.msg')

int_to_cell = {v: k for k, v in cell_to_int.items()}
cell_tidy_data.index = cell_tidy_data.index.map(int_to_cell)

## Gene Meta

In [12]:
gene_meta = pd.read_csv(
    '/home/hanliu/ref/mouse/gencode/vm22/gencode.vM22.annotation.gene.flat.tsv.gz',
    index_col='gene_id',
    sep='\t')

gene_to_int = {g: i for i, g in enumerate(gene_meta.index)}
gene_meta = gene_meta.reset_index()
gene_meta.index = gene_meta['gene_id'].map(gene_to_int)
print(gene_meta.shape)
gene_meta.head()

(55487, 22)


,gene_id,chrom,source,feature,start,end,score,strand,phase,transcript_id,...,gene_name,transcript_type,transcript_status,transcript_name,exon_number,exon_id,level,mgi_id,havana_gene,tag
gene_id,,,,,,,,,,,,,,,,,,,,,
0,ENSMUSG00000102693.1,chr1,HAVANA,gene,3073253,3074322,.,+,.,NaN,...,4933401J01Rik,NaN,NaN,NaN,NaN,NaN,2,MGI:1918292,OTTMUSG00000049935.1,NaN
1,ENSMUSG00000064842.1,chr1,ENSEMBL,gene,3102016,3102125,.,+,.,NaN,...,Gm26206,NaN,NaN,NaN,NaN,NaN,3,MGI:5455983,NaN,NaN
2,ENSMUSG00000051951.5,chr1,HAVANA,gene,3205901,3671498,.,-,.,NaN,...,Xkr4,NaN,NaN,NaN,NaN,NaN,2,MGI:3528744,OTTMUSG00000026353.2,NaN
3,ENSMUSG00000102851.1,chr1,HAVANA,gene,3252757,3253236,.,+,.,NaN,...,Gm18956,NaN,NaN,NaN,NaN,NaN,1,MGI:5011141,OTTMUSG00000049958.1,pseudo_consens
4,ENSMUSG00000103377.1,chr1,HAVANA,gene,3365731,3368549,.,-,.,NaN,...,Gm37180,NaN,NaN,NaN,NaN,NaN,2,MGI:5610408,OTTMUSG00000049960.1,NaN


In [20]:
use_columns = [
    'gene_id', 'chrom', 'start', 'end', 'strand', 'gene_type', 'gene_name',
    'level', 'mgi_id', 'tag'
]
gene_meta = gene_meta[use_columns].fillna('-')
gene_meta.index = 'gene_int'
gene_meta.to_csv('GeneMeta.csv.gz')

## Gene Rate

When doing analysis, we use cell chunks, but when loading gene data for visualization, we use gene chunks and load all cell, here it take some time to resave the data, but after that, load gene data for all cell is lightening fast

In [4]:
chunk_size = 20

In [5]:
gene_rate_mcds_list = list(
    pathlib.Path(
        '/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/'
    ).glob('GeneWithSlop2kb.gene_da_rate.*.mcds'))

In [6]:
mcds = xr.open_mfdataset(gene_rate_mcds_list,
                         concat_dim='cell',
                         combine='nested',
                         chunks={'gene': chunk_size})

mcds.coords['gene'] = mcds.get_index('gene').map(gene_to_int)

del mcds.coords['strand_type']
del mcds.coords['geneslop2k_end']
del mcds.coords['geneslop2k_start']
del mcds.coords['geneslop2k_chrom']

mcds

<xarray.Dataset>
Dimensions:  (cell: 104340, gene: 55487, mc_type: 2)
Coordinates:
  * gene     (gene) int64 0 1 2 3 4 5 6 ... 55481 55482 55483 55484 55485 55486
  * mc_type  (mc_type) object 'CGN' 'CHN'
  * cell     (cell) object '1A_M_0' '1A_M_1' ... '10F_M_997' '10F_M_998'
Data variables:
    gene_da  (cell, gene, mc_type) float64 dask.array<chunksize=(10000, 20, 2), meta=np.ndarray>

### Save chunks

In [7]:
save_chunk_size = 1000

gene_index = mcds.get_index('gene')


def load_and_save(genes, i):
    mcds_chunk = mcds.sel(gene=genes).load().sel(
        cell=pd.Index(cell_to_int.keys()))
    mcds_chunk.coords['cell'] = mcds_chunk.get_index('cell').map(cell_to_int)
    mcds_chunk.to_netcdf(
        f'GeneSlop2K.Bayes.Norm.CHCG.chunk{int(i//save_chunk_size)}.mcds')
    return


chunks = []
for i in range(0, gene_index.size, save_chunk_size):
    print('chunk', i)
    genes = gene_index[i:i + save_chunk_size]
    load_and_save(genes, i)

chunk 0
chunk 1000
chunk 2000
chunk 3000
chunk 4000
chunk 5000
chunk 6000
chunk 7000
chunk 8000
chunk 9000
chunk 10000
chunk 11000
chunk 12000
chunk 13000
chunk 14000
chunk 15000
chunk 16000
chunk 17000
chunk 18000
chunk 19000
chunk 20000
chunk 21000
chunk 22000
chunk 23000
chunk 24000
chunk 25000
chunk 26000
chunk 27000
chunk 28000
chunk 29000
chunk 30000
chunk 31000
chunk 32000
chunk 33000
chunk 34000
chunk 35000
chunk 36000
chunk 37000
chunk 38000
chunk 39000
chunk 40000
chunk 41000
chunk 42000
chunk 43000
chunk 44000
chunk 45000
chunk 46000
chunk 47000
chunk 48000
chunk 49000
chunk 50000
chunk 51000
chunk 52000
chunk 53000
chunk 54000
chunk 55000


### Gene to chunk name

In [21]:
records = {}
for i in range(0, gene_index.size, save_chunk_size):
    genes = gene_index[i:i + save_chunk_size]
    for g in genes:
        records[
            g] = f'GeneSlop2K.Bayes.Norm.CHCG.chunk{int(i//save_chunk_size)}.mcds'
with open('GeneToMCDSName.json', 'w') as f:
    json.dump(records, f)

### Load an example

In [38]:
%%timeit
xr.open_dataset('./GeneSlop2K.Bayes.Norm.CHCG.chunk20.mcds')['gene_da'].sel(
    gene=20001, mc_type='CHN').to_pandas()

258 ms ± 3.69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
